In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import backend as K


In [ ]:
# For reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Cell 2: Set up constants and directory paths
# Dataset paths
ROOT_DIR = "../data/MSFD"
TRAIN_DIR = os.path.join(ROOT_DIR, "1")
TEST_DIR = os.path.join(ROOT_DIR, "2")

# Image directories
TRAIN_IMG_DIR = os.path.join(TRAIN_DIR, "img")
TRAIN_FACE_CROP_DIR = os.path.join(TRAIN_DIR, "face_crop")
TRAIN_FACE_CROP_SEG_DIR = os.path.join(TRAIN_DIR, "face_crop_segmentation")
TEST_IMG_DIR = os.path.join(TEST_DIR, "img")

# Create directories for test results
TEST_FACE_CROP_DIR = os.path.join(TEST_DIR, "face_crop")
TEST_FACE_CROP_SEG_DIR = os.path.join(TEST_DIR, "face_crop_segmentation")
PREDICTIONS_DIR = os.path.join(TEST_DIR, "predictions")

In [ ]:
# Create directories if they don't exist
os.makedirs(TEST_FACE_CROP_DIR, exist_ok=True)
os.makedirs(TEST_FACE_CROP_SEG_DIR, exist_ok=True)
os.makedirs(PREDICTIONS_DIR, exist_ok=True)

# Model checkpoints directory
CHECKPOINTS_DIR = os.path.join(ROOT_DIR, "model_checkpoints")
os.makedirs(CHECKPOINTS_DIR, exist_ok=True)

# Model parameters
IMG_SIZE = 128  # Resize all images to 128x128
BATCH_SIZE = 16
EPOCHS = 50
LEARNING_RATE = 1e-4

In [ ]:
# Cell 3: Define IoU and Dice coefficient metrics
def iou_coef(y_true, y_pred, smooth=1):
    """Calculate IoU (Intersection over Union) coefficient."""
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(tf.cast(y_pred > 0.5, tf.float32))
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

def dice_coef(y_true, y_pred, smooth=1):
    """Calculate Dice coefficient."""
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(tf.cast(y_pred > 0.5, tf.float32))
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Define loss functions
def dice_loss(y_true, y_pred):
    """Dice loss function."""
    return 1 - dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    """Combined binary cross-entropy and dice loss."""
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)

In [ ]:
# Cell 4: Load and examine the dataset
# Load the CSV file containing bounding box information
df = pd.read_csv(os.path.join(TRAIN_DIR, "dataset.csv"))
print(f"Dataset shape: {df.shape}")
print("First 5 rows of the dataset:")
print(df.head())

# Check the distribution of masked and non-masked faces
mask_distribution = df['with_mask'].value_counts(normalize=True) * 100
print("\nDistribution of masked vs non-masked faces:")
print(mask_distribution)

# Visualize a sample image with its bounding box
def visualize_sample(index=0):
    row = df.iloc[index]
    img_path = os.path.join(TRAIN_IMG_DIR, row['filename'])
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Draw bounding box
    x1, y1, x2, y2 = row['x1'], row['y1'], row['x2'], row['y2']
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(img)
    plt.title(f"File: {row['filename']}, Mask: {row['with_mask']}")
    plt.axis('off')
    plt.show()
    
    # Find corresponding cropped face and segmentation
    face_id = row['filename'].split('.')[0]
    face_crop_files = [f for f in os.listdir(TRAIN_FACE_CROP_DIR) if f.startswith(face_id)]
    
    if face_crop_files:
        fig, axes = plt.subplots(len(face_crop_files), 2, figsize=(10, 4*len(face_crop_files)))
        if len(face_crop_files) == 1:
            axes = np.array([axes])
        
        for i, face_file in enumerate(face_crop_files):
            # Display cropped face
            face_img = cv2.imread(os.path.join(TRAIN_FACE_CROP_DIR, face_file))
            face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
            axes[i, 0].imshow(face_img)
            axes[i, 0].set_title(f"Cropped Face: {face_file}")
            axes[i, 0].axis('off')
            
            # Display segmentation mask
            seg_img = cv2.imread(os.path.join(TRAIN_FACE_CROP_SEG_DIR, face_file), cv2.IMREAD_GRAYSCALE)
            axes[i, 1].imshow(seg_img, cmap='gray')
            axes[i, 1].set_title(f"Segmentation: {face_file}")
            axes[i, 1].axis('off')
        
        plt.tight_layout()
        plt.show()

visualize_sample(3)  # Visualize sample with index 3 (can change to see other samples)

In [ ]:
# Cell 5: Preprocess the data - Create dataset for training
def get_face_crop_and_mask_paths():
    """Get paths of cropped faces and their corresponding masks."""
    face_files = sorted(os.listdir(TRAIN_FACE_CROP_DIR))
    mask_files = sorted(os.listdir(TRAIN_FACE_CROP_SEG_DIR))
    
    # Ensure that both directories have the same files
    common_files = set(face_files).intersection(set(mask_files))
    print(f"Found {len(common_files)} matching face crop and segmentation mask pairs.")
    
    face_paths = [os.path.join(TRAIN_FACE_CROP_DIR, f) for f in face_files if f in common_files]
    mask_paths = [os.path.join(TRAIN_FACE_CROP_SEG_DIR, f) for f in mask_files if f in common_files]
    
    return face_paths, mask_paths

def preprocess_data():
    """Preprocess images and masks for the U-Net model."""
    face_paths, mask_paths = get_face_crop_and_mask_paths()
    
    # Split into training and validation sets
    train_face_paths, val_face_paths, train_mask_paths, val_mask_paths = train_test_split(
        face_paths, mask_paths, test_size=0.2, random_state=42
    )
    
    print(f"Training samples: {len(train_face_paths)}")
    print(f"Validation samples: {len(val_face_paths)}")
    
    return train_face_paths, val_face_paths, train_mask_paths, val_mask_paths

# Get training and validation paths
train_face_paths, val_face_paths, train_mask_paths, val_mask_paths = preprocess_data()


In [ ]:
# Cell 6: Data Generator for training
class DataGenerator(tf.keras.utils.Sequence):
    """Data generator for training and validation."""
    def __init__(self, face_paths, mask_paths, batch_size=16, img_size=128, augment=False, shuffle=True):
        self.face_paths = face_paths
        self.mask_paths = mask_paths
        self.batch_size = batch_size
        self.img_size = img_size
        self.augment = augment
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.face_paths))
        
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        """Return the number of batches per epoch."""
        return int(np.ceil(len(self.face_paths) / self.batch_size))
    
    def __getitem__(self, idx):
        """Generate one batch of data."""
        batch_indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        batch_faces = [self.face_paths[i] for i in batch_indexes]
        batch_masks = [self.mask_paths[i] for i in batch_indexes]
        
        X, y = self.__data_generation(batch_faces, batch_masks)
        
        return X, y
    
    def on_epoch_end(self):
        """Update indexes after each epoch."""
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __data_generation(self, batch_faces, batch_masks):
        """Generate batches of augmented data."""
        X = np.empty((len(batch_faces), self.img_size, self.img_size, 3), dtype=np.float32)
        y = np.empty((len(batch_faces), self.img_size, self.img_size, 1), dtype=np.float32)
        
        for i, (face_path, mask_path) in enumerate(zip(batch_faces, batch_masks)):
            # Load and preprocess face image
            face = cv2.imread(face_path)
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (self.img_size, self.img_size))
            
            # Load and preprocess mask image
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (self.img_size, self.img_size))
            mask = np.expand_dims(mask, axis=-1)
            
            # Normalize images
            face = face / 255.0
            mask = mask / 255.0
            
            # Apply augmentation if enabled (can add more augmentation techniques)
            if self.augment:
                # Random horizontal flip
                if np.random.random() > 0.5:
                    face = np.fliplr(face)
                    mask = np.fliplr(mask)
                
                # Random brightness adjustment
                if np.random.random() > 0.5:
                    factor = 0.2 + np.random.uniform(0, 0.8)
                    face = face * factor
                    face = np.clip(face, 0, 1.0)
            
            X[i,] = face
            y[i,] = mask
        
        return X, y

# Create data generators
train_generator = DataGenerator(
    train_face_paths, 
    train_mask_paths,
    batch_size=BATCH_SIZE,
    img_size=IMG_SIZE,
    augment=True
)

val_generator = DataGenerator(
    val_face_paths, 
    val_mask_paths,
    batch_size=BATCH_SIZE,
    img_size=IMG_SIZE,
    augment=False
)

# Visualize a few training samples
def visualize_batch_samples():
    X_batch, y_batch = train_generator[0]
    
    fig, axes = plt.subplots(4, 2, figsize=(10, 16))
    for i in range(4):
        # Display image
        axes[i, 0].imshow(X_batch[i])
        axes[i, 0].set_title(f"Face Image {i+1}")
        axes[i, 0].axis('off')
        
        # Display mask
        axes[i, 1].imshow(y_batch[i].squeeze(), cmap='gray')
        axes[i, 1].set_title(f"Mask {i+1}")
        axes[i, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

visualize_batch_samples()


In [ ]:
# Cell 7: Define U-Net Model Architecture
def build_unet_model(input_size=(IMG_SIZE, IMG_SIZE, 3)):
    """Build the U-Net model architecture."""
    # Input layer
    inputs = Input(input_size)
    
    # Encoder (downsampling path)
    # Block 1
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # Block 2
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Block 3
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    # Block 4
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    
    # Bridge
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder (upsampling path)
    # Block 6
    up6 = Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(drop5)
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)
    
    # Block 7
    up7 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)
    
    # Block 8
    up8 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)
    
    # Block 9
    up9 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv8)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    
    # Output layer
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    # Create and compile model
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

# Cell 8: Build and compile the model
def compile_model(model):
    """Compile the model with optimizers and loss functions."""
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss=bce_dice_loss,
        metrics=[dice_coef, iou_coef, 'accuracy']
    )
    return model

# Build and compile the U-Net model
model = build_unet_model(input_size=(IMG_SIZE, IMG_SIZE, 3))
model = compile_model(model)

# Print model summary
model.summary()

In [ ]:
# Cell 9: Define callbacks for training
# Callbacks for model training
callbacks = [
    # Early stopping to prevent overfitting
    EarlyStopping(
        monitor='val_loss', 
        patience=10, 
        verbose=1, 
        restore_best_weights=True
    ),
    
    # Reduce learning rate when a metric has stopped improving
    ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.5, 
        patience=5, 
        verbose=1, 
        min_lr=1e-6
    ),
    
    # Save model checkpoints
    ModelCheckpoint(
        filepath=os.path.join(CHECKPOINTS_DIR, 'mask_unet_model.h5'),
        save_best_only=True,
        monitor='val_dice_coef',
        mode='max',
        verbose=1
    )
]



In [ ]:
# Cell 10: Train the model
# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

# Save the final model
model.save(os.path.join(CHECKPOINTS_DIR, 'mask_unet_final_model.h5'))



In [ ]:
# Cell 11: Evaluate model performance and visualize training metrics
# Plot training history
def plot_training_history(history):
    """Plot training and validation metrics."""
    # Plot loss
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot dice coefficient
    plt.subplot(1, 3, 2)
    plt.plot(history.history['dice_coef'], label='Training Dice')
    plt.plot(history.history['val_dice_coef'], label='Validation Dice')
    plt.title('Dice Coefficient')
    plt.xlabel('Epoch')
    plt.ylabel('Dice')
    plt.legend()
    
    # Plot IoU coefficient
    plt.subplot(1, 3, 3)
    plt.plot(history.history['iou_coef'], label='Training IoU')
    plt.plot(history.history['val_iou_coef'], label='Validation IoU')
    plt.title('IoU Coefficient')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

# Cell 12: Evaluate model on validation set
# Evaluate the model on the validation set
val_loss, val_dice, val_iou, val_acc = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Dice Coefficient: {val_dice:.4f}")
print(f"Validation IoU: {val_iou:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")

# Visualize predictions on validation samples
def visualize_predictions(sample_count=5):
    """Visualize model predictions on validation samples."""
    # Get a batch from validation generator
    X_val, y_val = next(iter(val_generator))
    
    # Make predictions
    y_pred = model.predict(X_val)
    
    # Show sample_count predictions
    n = min(sample_count, len(X_val))
    fig, axes = plt.subplots(n, 3, figsize=(15, 5*n))
    
    for i in range(n):
        # Display original image
        axes[i, 0].imshow(X_val[i])
        axes[i, 0].set_title("Original Image")
        axes[i, 0].axis('off')
        
        # Display ground truth mask
        axes[i, 1].imshow(y_val[i].squeeze(), cmap='gray')
        axes[i, 1].set_title("Ground Truth Mask")
        axes[i, 1].axis('off')
        
        # Display predicted mask
        axes[i, 2].imshow(y_pred[i].squeeze(), cmap='gray')
        axes[i, 2].set_title(f"Predicted Mask\nDice: {dice_coef(y_val[i], y_pred[i]).numpy():.4f}")
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.show()

visualize_predictions(5)

In [ ]:
# Cell 13: Hyperparameter tuning (basic example)
def hyperparameter_tuning():
    """Simple hyperparameter tuning for U-Net."""
    # Define hyperparameter combinations to try
    learning_rates = [1e-3, 1e-4]
    batch_sizes = [8, 16]
    
    # Results dictionary
    results = []
    
    for lr in learning_rates:
        for bs in batch_sizes:
            print(f"\nTesting LR={lr}, Batch Size={bs}")
            
            # Create generators with current batch size
            train_gen = DataGenerator(
                train_face_paths[:100],  # Use subset for quick tuning
                train_mask_paths[:100],
                batch_size=bs,
                img_size=IMG_SIZE,
                augment=True
            )
            
            val_gen = DataGenerator(
                val_face_paths[:50],  # Use subset for quick tuning
                val_mask_paths[:50],
                batch_size=bs,
                img_size=IMG_SIZE,
                augment=False
            )
            
            # Build and compile model with current learning rate
            model = build_unet_model()
            model.compile(
                optimizer=Adam(learning_rate=lr),
                loss=bce_dice_loss,
                metrics=[dice_coef, iou_coef]
            )
            
            # Train for a few epochs
            history = model.fit(
                train_gen,
                epochs=5,  # Few epochs for quick evaluation
                validation_data=val_gen,
                verbose=1
            )
            
            # Get final validation metrics
            val_loss = history.history['val_loss'][-1]
            val_dice = history.history['val_dice_coef'][-1]
            val_iou = history.history['val_iou_coef'][-1]
            
            # Store results
            results.append({
                'learning_rate': lr,
                'batch_size': bs,
                'val_loss': val_loss,
                'val_dice': val_dice,
                'val_iou': val_iou
            })
    
    # Convert to DataFrame and find best combination
    results_df = pd.DataFrame(results)
    print("\nHyperparameter Tuning Results:")
    print(results_df)
    
    best_combo = results_df.loc[results_df['val_dice'].idxmax()]
    print(f"\nBest Hyperparameters:\nLearning Rate: {best_combo['learning_rate']}\nBatch Size: {best_combo['batch_size']}")
    print(f"Best Validation Dice: {best_combo['val_dice']:.4f}")
    
    return best_combo['learning_rate'], best_combo['batch_size']

# Comment out if you don't want to run hyperparameter tuning
# best_lr, best_bs = hyperparameter_tuning()